In [1]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from string import punctuation
punct = punctuation+'©«»—…“”*№–'

In [2]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [3]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [4]:
df.head(10)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь
3,"«Сачхере» — «Абердин» И тот, и другой клуб оде...",1.45,Айнтрахт - 3 и более гола,Лига Ставок,2019-07-25 17:00:00,Лига Европы УЕФА,Флора,Айнтрахт Франкфурт
4,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
5,Июль подходит к концу. Скоро мадридское дерби ...,1.62,Атлетико забьет 1 или 2 гола: да,Фонбет,2019-07-27 00:06:00,Международный Кубок чемпионов,Реал Мадрид,Атлетико Мадрид
6,"«Ахмат», который доставил много проблем команд...",1.71,Ахмат забьет 1 или 2 гола: да,Пари-Матч,2019-07-14 18:30:00,Премьер-Лига,Ахмат,Краснодар
7,"Никто толком не представляет, в каких составах...",2.45,Бавария забьет в обоих таймах,Лига Ставок,2019-07-18 03:00:00,Международный Кубок чемпионов,Арсенал,Бавария
8,"Матч вроде бы товарищеский, но - это и репутац...",3.25,Барселона забьет в каждом тайме,Лига Ставок,2019-07-23 10:30:00,Клубные товарищеские матчи,Барселона,Челси
9,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония


In [5]:
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'
    
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))
    
df['Language'] = lang
grouped=df.groupby('Language')
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)

russian.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь


In [6]:
unique_leagues = russian['league_name_in_russian'].unique().tolist()
len(unique_leagues)

288

In [7]:
unique_teams1 = russian['team_of_season_1_name_in_russian'].unique().tolist()
len(unique_teams1)

3773

In [8]:
unique_teams2 = russian['team_of_season_2_name_in_russian'].unique().tolist()
len(unique_teams2)

3730

In [9]:
unique_teams = unique_teams1 + unique_teams2
len(set(unique_teams))

4422

In [10]:
russian['result.tsv'][5133]

'Швейцария - Англия Совершенно бестолковый матч. Команды без особой мотивации должны мучиться на жаре днем. Ну раз уж приехали в Португалию, чего бы не сыграть, по сути, товарищеский матч. Думаю, у Швейцарии преимущество должно быть. Англия играла на день позже, ей пришлось еще и овертайм провести. Плюс результат ударил по амбициям команды Саутгейта. Она упустила победу в основное время. Ее гол отменили из-за микроскопического офсайда. Ну и сами игроки наошибались от души. Швейцария проиграла тоже обидно. Но было больше времени, чтобы все это переварить. Ну и день отдыха лишний. К тому же, как мне кажется, для Швейцарии престижнее занять третье место в первой Лиге наций. Британцы же максималисты. Да и по качеству футбола Швейцария понравилась больше. Ей бы еще решить вопрос с мотивацией. Думаю, пока британцы будут разбираться со своим настроем, под шумок швейцарцы могут забить. Тем более в обороне англичан наверняка будет много перестановок. Нет смысла нагружать основных ребят, конец с

In [12]:
russian = russian.reset_index(drop=True)

### Команда 1 и 2 (список + прямой мэтч)

In [20]:
teams1_match = np.zeros(df.shape[0])
teams2_match = np.zeros(df.shape[0])

https://stackoverflow.com/questions/15852436/how-to-find-the-2nd-max-of-a-counter-python

In [22]:
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for txt in txt_tokens:
        count_tokens = []
        for token in txt:
            token = token.strip(punct)
            #print(token)
            if token in unique_teams:
                count_tokens.append(token)
        #print(count_tokens)
        c = Counter(count_tokens)
        #if count_tokens == 2
        #print(df['team_of_season_1_name_in_russian'][i], df['team_of_season_2_name_in_russian'][i])
        #print(c)
        #print(max(c))
        #print(c.most_common()[0:2])
        #print(text)
        try:
            max_w1 = sorted(c, key=c.get)[-1]
            #print(max_w1)
        except:
            continue
        try:
            max_w2 = sorted(c, key=c.get)[-2]
            #print(max_w2)
        except:
            continue
        
        #print('мэтч команды 1')
        if df['team_of_season_1_name_in_russian'][i] == max_w1 or df['team_of_season_1_name_in_russian'][i] == max_w2:
            teams1_match[i] = 1
            #print(df['team_of_season_1_name_in_russian'][i], max_w1)
            #print(df['team_of_season_1_name_in_russian'][i], max_w2)
        
        #print('мэтч команды 2')
        if df['team_of_season_2_name_in_russian'][i] == max_w1 or df['team_of_season_2_name_in_russian'][i] == max_w2:
            teams2_match[i] = 1
            #print(df['team_of_season_2_name_in_russian'][i], max_w1)
            #print(df['team_of_season_2_name_in_russian'][i], max_w2)
        
        

702908it [5:24:59, 36.05it/s] 


In [25]:
df['list_direct_match_for_teams1'] = teams1_match

In [26]:
df['list_direct_match_for_teams2'] = teams2_match

In [27]:
success1 = df[df['list_direct_match_for_teams1'] == 1].shape[0]
success2 = df[df['list_direct_match_for_teams2'] == 1].shape[0]

In [29]:
all_vals = df['list_direct_match_for_teams1'].shape[0]

In [30]:
result_team1 = success1 / all_vals
print(result_team1)

0.20481058687623416


In [31]:
result_team2 = success2 / all_vals
print(result_team2)

0.2051363763109824


### Лиги

In [33]:
leagues_match = np.zeros(df.shape[0])

In [ ]:
%%time 
for i, text in tqdm(enumerate(df['result.tsv'].values)):
    n_words_league = len(word_tokenize(df['league_name_in_russian'][i]))
    N = n_words_league
    words = word_tokenize(text)
    words = [word.lower() for word in words if word not in punct ]
    grams = [words[i:i+N] for i in range(len(words)-N+1)]

    for gram in grams: 
        count_tokens = []
        str1 = " "
        joined_gram = str1.join(gram)
        #print(joined_gram)
        if joined_gram in unique_leagues:
            count_tokens.append(joined_gram)
    
    c = Counter(count_tokens)
    
    try:
        max_w1 = sorted(c, key=c.get)[-1]
        #print(max_w1)
        if df['league_name_in_russian'][i].lower() == max_w1:
            leagues_match[i] = 1
            #print(df['league_name_in_russian'][i].lower(), max_w1)
        
    except:
        continue

In [35]:
df['list_direct_match_for_leagues'] = leagues_match

In [36]:
success = df[df['list_direct_match_for_leagues'] == 1].shape[0]

In [37]:
all_vals = df['list_direct_match_for_leagues'].shape[0]

In [38]:
result_leagues = success / all_vals
print(result_leagues)

0.0


### русский

In [26]:
teams1_match = np.zeros(russian.shape[0])
teams2_match = np.zeros(russian.shape[0])

In [27]:
for i, text in enumerate(russian['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for txt in txt_tokens:
        count_tokens = []
        for token in txt:
            token = token.strip(punct)
            #print(token)
            if token in unique_teams:
                count_tokens.append(token)
        #print(count_tokens)
        c = Counter(count_tokens)
        #if count_tokens == 2
        #print(df['team_of_season_1_name_in_russian'][i], df['team_of_season_2_name_in_russian'][i])
        #print(c)
        #print(max(c))
        #print(c.most_common()[0:2])
        #print(text)
        
        try: 
            max_w1 = sorted(c, key=c.get)[-1]
            #print(max_w1)
        except:
            continue
        
        try:
            max_w2 = sorted(c, key=c.get)[-2]
            #print(max_w2)
        except:
            continue
        
        
#         print('мэтч команды 1')
        
        if russian['team_of_season_1_name_in_russian'][i] == max_w1 or russian['team_of_season_1_name_in_russian'][i] == max_w2:
            teams1_match[i] = 1
#             print(russian['team_of_season_1_name_in_russian'][i], max_w1)
#             print(russian['team_of_season_1_name_in_russian'][i], max_w2)
        
#         print('мэтч команды 2')
        
        if russian['team_of_season_2_name_in_russian'][i] == max_w1 or russian['team_of_season_2_name_in_russian'][i] == max_w2:
            teams2_match[i] = 1
#             print(russian['team_of_season_2_name_in_russian'][i], max_w1)
#             print(russian['team_of_season_2_name_in_russian'][i], max_w2)


In [28]:
russian['list_direct_match_for_teams1'] = teams1_match
russian['list_direct_match_for_teams2'] = teams2_match

In [29]:
success1 = russian[russian['list_direct_match_for_teams1'] == 1].shape[0]
success2 = russian[russian['list_direct_match_for_teams2'] == 1].shape[0]

all_vals = russian['list_direct_match_for_teams1'].shape[0]

result_team1 = success1 / all_vals
print(result_team1)

result_team2 = success2 / all_vals
print(result_team2)

0.24154306454941932
0.2419223030854243


In [30]:
russian.to_csv(r'C:\Users\User\Desktop\project\russian_lists_and_directmatch.csv', index = False)

In [31]:
unique_leagues[:10]

['Международные товарищеские матчи',
 'Кубок Америки',
 'Лига Европы УЕФА',
 'Международный Кубок чемпионов',
 'Премьер-Лига',
 'Клубные товарищеские матчи',
 'Чемпионат Европы. Квалификация',
 'Эредивизи',
 'Чемпионшип',
 'Премьер-лига']

In [32]:
leagues_match = np.zeros(russian.shape[0])

In [39]:
%%time 
for i, text in tqdm(enumerate(russian['result.tsv'].values)):
    try:
        n_words_league = len(word_tokenize(russian['league_name_in_russian'][i]))
        N = n_words_league
        words = word_tokenize(text)
        words = [word.lower() for word in words if word not in punct ]
        grams = [words[i:i+N] for i in range(len(words)-N+1)]
        #print(grams)
    
        for gram in grams: 
            count_tokens = []
            str1 = " "
            joined_gram = str1.join(gram)
            #print(joined_gram)
            if joined_gram in unique_leagues:
                count_tokens.append(joined_gram)
    
        c = Counter(count_tokens)
        #print(c)
    
        try:
            max_w1 = sorted(c, key=c.get)[-1]
            #print(max_w1)
            #print(russian['league_name_in_russian'][i].lower(), max_w1)

            if russian['league_name_in_russian'][i].lower() == max_w1:
                leagues_match[i] = 1
                #print(russian['league_name_in_russian'][i].lower(), max_w1)
        
        except:
            continue
    except:
        continue



0it [00:00, ?it/s]

5it [00:00, 45.90it/s]

10it [00:00, 43.06it/s]

16it [00:00, 46.18it/s]

22it [00:00, 49.60it/s]

30it [00:00, 54.41it/s]

37it [00:00, 57.04it/s]

43it [00:00, 56.10it/s]

49it [00:00, 48.13it/s]

54it [00:01, 42.48it/s]

59it [00:01, 42.41it/s]

65it [00:01, 44.97it/s]

70it [00:01, 44.82it/s]

76it [00:01, 48.15it/s]

81it [00:01, 48.40it/s]

87it [00:01, 48.85it/s]

93it [00:01, 50.22it/s]

99it [00:02, 45.07it/s]

104it [00:02, 39.07it/s]

109it [00:02, 35.73it/s]

114it [00:02, 38.99it/s]

119it [00:02, 41.14it/s]

126it [00:02, 45.77it/s]

131it [00:02, 45.39it/s]

136it [00:02, 43.70it/s]

143it [00:03, 48.31it/s]

149it [00:03, 45.40it/s]

154it [00:03, 44.60it/s]

159it [00:03, 43.94it/s]

164it [00:03, 43.67it/s]

169it [00:03, 38.52it/s]

174it [00:03, 38.22it/s]

178it [00:03, 38.03it/s]

183it [00:04, 40.88it/s]

189it [00:04, 42.79it/s]

194it [00:04, 40.79it/s]

199it [00:04, 38.28it/s]

203it [00:04, 36.39it/s]

208it [00:04, 37.34it/s]

212it [0

19485it [01:23, 251.96it/s]

19511it [01:23, 252.27it/s]

19542it [01:23, 257.68it/s]

19568it [01:23, 250.45it/s]

19594it [01:24, 236.94it/s]

19618it [01:24, 235.04it/s]

19646it [01:24, 241.24it/s]

19671it [01:24, 237.29it/s]

19703it [01:24, 252.41it/s]

19729it [01:24, 248.13it/s]

19758it [01:24, 253.96it/s]

19784it [01:24, 245.79it/s]

19813it [01:24, 252.28it/s]

19843it [01:25, 255.04it/s]

19870it [01:25, 257.43it/s]

19900it [01:25, 260.52it/s]

19927it [01:25, 250.81it/s]

19953it [01:25, 244.42it/s]

19978it [01:25, 116.82it/s]

19997it [01:26, 109.50it/s]

20014it [01:26, 104.14it/s]

20029it [01:26, 88.93it/s] 

20041it [01:26, 88.25it/s]

20052it [01:26, 80.14it/s]

20062it [01:26, 81.33it/s]

20072it [01:27, 76.47it/s]

20081it [01:27, 76.81it/s]

20090it [01:27, 76.76it/s]

20099it [01:27, 79.45it/s]

20109it [01:27, 82.59it/s]

20118it [01:27, 61.75it/s]

20126it [01:27, 62.90it/s]

20136it [01:28, 67.97it/s]

20146it [01:28, 73.55it/s]

20159it [01:28, 83.08it/s]

114079it [02:47, 273.31it/s]

114108it [02:47, 253.27it/s]

114142it [02:47, 273.94it/s]

114172it [02:47, 277.26it/s]

114215it [02:47, 304.44it/s]

114260it [02:47, 336.00it/s]

114296it [02:48, 329.83it/s]

114331it [02:48, 220.95it/s]

114359it [02:48, 187.89it/s]

114383it [02:48, 183.43it/s]

114405it [02:48, 177.07it/s]

114495it [02:48, 229.29it/s]

114544it [02:49, 269.71it/s]

114603it [02:49, 317.09it/s]

114670it [02:49, 371.84it/s]

114725it [02:49, 409.39it/s]

114844it [02:49, 500.73it/s]

114934it [02:49, 567.96it/s]

115028it [02:49, 635.44it/s]

115136it [02:49, 709.34it/s]

115245it [02:49, 783.83it/s]

115347it [02:50, 833.30it/s]

115439it [02:50, 826.63it/s]

115528it [02:50, 553.89it/s]

115600it [02:50, 479.55it/s]

115661it [02:50, 468.93it/s]

115717it [02:50, 469.57it/s]

115797it [02:50, 533.02it/s]

115858it [02:51, 447.81it/s]

115911it [02:51, 439.64it/s]

115961it [02:51, 386.34it/s]

116005it [02:51, 353.36it/s]

116045it [02:51, 299.04it/s]

116087it [

132471it [04:04, 68.67it/s]

132479it [04:04, 70.22it/s]

132487it [04:04, 67.45it/s]

132494it [04:04, 65.90it/s]

132502it [04:04, 68.63it/s]

132512it [04:05, 75.09it/s]

132521it [04:05, 77.88it/s]

132529it [04:05, 75.79it/s]

132537it [04:05, 69.67it/s]

132545it [04:05, 69.25it/s]

132553it [04:05, 71.11it/s]

132561it [04:05, 58.31it/s]

132568it [04:06, 53.85it/s]

132577it [04:06, 60.13it/s]

132586it [04:06, 65.97it/s]

132594it [04:06, 66.95it/s]

132602it [04:06, 64.07it/s]

132609it [04:06, 63.94it/s]

132616it [04:06, 56.75it/s]

132622it [04:06, 56.70it/s]

132630it [04:06, 58.77it/s]

132641it [04:07, 67.75it/s]

132650it [04:07, 69.55it/s]

132658it [04:07, 58.07it/s]

132670it [04:07, 68.07it/s]

132686it [04:07, 80.77it/s]

132701it [04:07, 93.32it/s]

132728it [04:07, 114.07it/s]

132749it [04:07, 130.69it/s]

132773it [04:08, 148.03it/s]

132793it [04:08, 158.51it/s]

132814it [04:08, 170.70it/s]

132835it [04:08, 180.13it/s]

132862it [04:08, 196.65it/s]

132884i

137602it [04:35, 159.63it/s]

137624it [04:35, 170.15it/s]

137646it [04:35, 175.35it/s]

137667it [04:35, 182.72it/s]

137690it [04:35, 190.29it/s]

137710it [04:35, 183.19it/s]

137729it [04:35, 169.00it/s]

137747it [04:35, 171.75it/s]

137765it [04:35, 159.12it/s]

137784it [04:36, 165.99it/s]

137802it [04:36, 164.40it/s]

137821it [04:36, 169.40it/s]

137840it [04:36, 168.89it/s]

137858it [04:36, 170.02it/s]

137876it [04:36, 167.62it/s]

137894it [04:36, 168.29it/s]

137915it [04:36, 174.52it/s]

137935it [04:36, 178.91it/s]

137957it [04:37, 184.33it/s]

137976it [04:37, 177.02it/s]

137994it [04:37, 174.56it/s]

138018it [04:37, 183.32it/s]

138037it [04:37, 180.17it/s]

138058it [04:37, 184.67it/s]

138081it [04:37, 190.34it/s]

138101it [04:37, 182.84it/s]

138120it [04:37, 179.65it/s]

138142it [04:38, 188.50it/s]

138162it [04:38, 184.88it/s]

138181it [04:38, 174.71it/s]

138201it [04:38, 181.19it/s]

138220it [04:38, 176.25it/s]

138239it [04:38, 179.64it/s]

138258it [

...

552366it [34:15, 151.91it/s]

552383it [34:16, 152.72it/s]

552399it [34:16, 148.40it/s]

552414it [34:16, 146.46it/s]

552430it [34:16, 148.37it/s]

552445it [34:16, 143.10it/s]

552461it [34:16, 145.78it/s]

552483it [34:16, 161.83it/s]

552500it [34:16, 154.03it/s]

552520it [34:16, 162.10it/s]

552537it [34:16, 162.64it/s]

552559it [34:17, 169.61it/s]

552582it [34:17, 178.11it/s]

552601it [34:17, 181.33it/s]

552620it [34:17, 181.45it/s]

552639it [34:17, 183.44it/s]

552658it [34:17, 180.63it/s]

552677it [34:17, 180.25it/s]

552696it [34:17, 150.49it/s]

552714it [34:18, 156.24it/s]

552731it [34:18, 155.76it/s]

552749it [34:18, 161.90it/s]

552771it [34:18, 173.48it/s]

552790it [34:18, 170.72it/s]

552811it [34:18, 180.54it/s]

552830it [34:18, 171.25it/s]

552848it [34:18, 169.40it/s]

552866it [34:18, 171.95it/s]

552886it [34:18, 172.51it/s]

552904it [34:19, 174.27it/s]

552922it [34:19, 175.91it/s]

552940it [34:19, 174.07it/s]

552958it [34:19, 170.79it/s]

552976it [

562497it [35:16, 166.88it/s]

562520it [35:16, 180.83it/s]

562539it [35:16, 175.89it/s]

562557it [35:16, 172.87it/s]

562576it [35:16, 172.23it/s]

562594it [35:16, 172.98it/s]

562613it [35:16, 176.45it/s]

562636it [35:16, 184.54it/s]

562655it [35:17, 179.81it/s]

562674it [35:17, 179.73it/s]

562694it [35:17, 185.29it/s]

562713it [35:17, 171.67it/s]

562731it [35:17, 164.37it/s]

562749it [35:17, 168.30it/s]

562767it [35:17, 171.63it/s]

562789it [35:17, 182.11it/s]

562811it [35:17, 186.51it/s]

562830it [35:18, 186.44it/s]

562849it [35:18, 171.66it/s]

562871it [35:18, 183.05it/s]

562891it [35:18, 172.09it/s]

562909it [35:18, 160.47it/s]

562927it [35:18, 163.09it/s]

562948it [35:18, 169.18it/s]

562966it [35:18, 164.14it/s]

562983it [35:19, 165.55it/s]

563000it [35:19, 151.90it/s]

563016it [35:19, 150.25it/s]

563032it [35:19, 150.23it/s]

563048it [35:19, 144.87it/s]

563070it [35:19, 157.16it/s]

563087it [35:19, 150.10it/s]

563111it [35:19, 166.19it/s]

563133it [

572507it [36:17, 159.88it/s]

572524it [36:17, 157.39it/s]

572544it [36:17, 168.12it/s]

572563it [36:17, 165.22it/s]

572580it [36:17, 161.73it/s]

572601it [36:17, 172.42it/s]

572619it [36:17, 172.18it/s]

572638it [36:17, 172.21it/s]

572656it [36:18, 171.07it/s]

572676it [36:18, 174.48it/s]

572694it [36:18, 175.08it/s]

572712it [36:18, 168.20it/s]

572729it [36:18, 162.31it/s]

572746it [36:18, 163.64it/s]

572763it [36:18, 157.35it/s]

572783it [36:18, 161.81it/s]

572803it [36:18, 171.53it/s]

572821it [36:19, 168.22it/s]

572839it [36:19, 168.86it/s]

572857it [36:19, 164.10it/s]

572874it [36:19, 159.12it/s]

572893it [36:19, 166.17it/s]

572910it [36:19, 164.42it/s]

572927it [36:19, 160.86it/s]

572944it [36:19, 160.26it/s]

572961it [36:19, 159.42it/s]

572977it [36:20, 156.38it/s]

572995it [36:20, 158.40it/s]

573011it [36:20, 158.51it/s]

573028it [36:20, 159.98it/s]

573045it [36:20, 161.50it/s]

573062it [36:20, 159.56it/s]

573078it [36:20, 156.58it/s]

573096it [

584219it [37:18, 249.58it/s]

584246it [37:18, 232.96it/s]

584274it [37:18, 244.72it/s]

584300it [37:18, 236.12it/s]

584325it [37:18, 205.17it/s]

584359it [37:19, 225.79it/s]

584386it [37:19, 231.12it/s]

584428it [37:19, 263.13it/s]

584457it [37:19, 259.52it/s]

584485it [37:19, 241.25it/s]

584521it [37:19, 257.47it/s]

584558it [37:19, 274.93it/s]

584596it [37:19, 297.39it/s]

584657it [37:19, 344.11it/s]

584696it [37:20, 354.93it/s]

584735it [37:20, 337.82it/s]

584771it [37:20, 287.60it/s]

584803it [37:20, 292.05it/s]

584837it [37:20, 295.98it/s]

584868it [37:20, 284.46it/s]

584904it [37:20, 303.49it/s]

584963it [37:20, 346.29it/s]

585001it [37:21, 322.55it/s]

585036it [37:21, 279.77it/s]

585067it [37:21, 281.69it/s]

585103it [37:21, 295.21it/s]

585134it [37:21, 276.52it/s]

585163it [37:21, 275.14it/s]

585192it [37:21, 265.83it/s]

585220it [37:21, 256.95it/s]

585258it [37:22, 280.23it/s]

585288it [37:22, 266.82it/s]

585316it [37:22, 267.07it/s]

585344it [

Wall time: 38min 11s


In [40]:
russian['list_direct_match_for_leagues'] = leagues_match
success1 = russian[russian['list_direct_match_for_leagues'] == 1].shape[0]
all_vals = russian['list_direct_match_for_leagues'].shape[0]

result_league = success1 / all_vals
print(result_league)


0.0


In [41]:
russian.to_csv(r'C:\Users\User\Desktop\project\russian_lists_and_directmatch_with_leagues.csv', index = False)